In [1]:
import re
import os
import time
import torch
import argparse
from tqdm import tqdm
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel, LoraConfig
from vllm import LLM, SamplingParams


merged_model_path = "/mnt/cfs/SPEECH/hupeng/github/llm_pp/vllm_models/g2ppp_qwen1half_7b_chat_prompt_v1_ckt310"

llm = LLM(model=merged_model_path, 
          enable_prefix_caching=True
         )

print(f"load merged model sucess!")

sampling_params = SamplingParams(
    stop=["<|endoftext|>","<|im_end|>","<|im_start|>"],
    temperature=0,
    max_tokens=64)

text_start = """<|im_start|>system
对提供的中文文本根据下面规则进行处理，来完成韵律标记和音节任务，文字标点符号和顺序保持不变。韵律标记：为句子中的每个字添加韵律标签，只有3类标签'#1,#2,#3'。规则如下：词：成词的两个或多个字的最后一个字后面标记为#1；短语：短语中的最后一个字后面标记为#2；如果字后面跟随标点符号（且该字不是句子的最后一个字），或者单句无标点符号文本并且文本过长，需要根据语意进行切分停顿，则该字的标记为#3。音节标记：给用户提供的文本中的中文标注音节，注意有的字是多音字，需要给出正确的读音’。例如：question1:‘卡尔普陪外孙玩滑梯’，answer1:‘ka2 er2 pu3 #2 pei2 #1 uai4 suen1 #1 uan2 hua2 ti1 #3。’；qeusion2:‘而部分网民也乐于围观起哄’，answer2:‘er2 #2 bu4 fen5 #1 uang3 min2 #2 ie3 #1 le4 v2 #1 uei2 guan1 #1 qi3 hong4 #3’；question3:‘舞池璀璨乐声欢快光影交错’，answer3:‘u3 chiii2 #1 cuei3 can4 #3 ve4 sheng1 #1 huan1 kuai4 #3 guang1 ing3 #1 jiao1 cuo4 #3’
<|im_end|>
<|im_start|>user
"""

text_end = """<|im_end|>
<|im_start|>assistant"""



/mnt/cfs/SPEECH/hupeng/tools/env/miniconda3/envs/llm_pp/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-07-23 23:45:58,257	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


INFO 07-23 23:45:58 llm_engine.py:100] Initializing an LLM engine (v0.4.2) with config: model='/mnt/cfs/SPEECH/hupeng/github/llm_pp/vllm_models/g2ppp_qwen1half_7b_chat_prompt_v1_ckt310', speculative_config=None, tokenizer='/mnt/cfs/SPEECH/hupeng/github/llm_pp/vllm_models/g2ppp_qwen1half_7b_chat_prompt_v1_ckt310', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.float16, max_seq_len=32768, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto, quantization_param_path=None, device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='outlines'), seed=0, served_model_name=/mnt/cfs/SPEECH/hupeng/github/llm_pp/vllm_models/g2ppp_qwen1half_7b_chat_prompt_v1_ckt310)


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


INFO 07-23 23:45:58 utils.py:660] Found nccl from library /usr/lib/x86_64-linux-gnu/libnccl.so.2
INFO 07-23 23:45:58 selector.py:27] Using FlashAttention-2 backend.
INFO 07-23 23:46:37 model_runner.py:175] Loading model weights took 14.3919 GB
INFO 07-23 23:46:40 gpu_executor.py:114] # GPU blocks: 6843, # CPU blocks: 512
INFO 07-23 23:46:41 model_runner.py:937] Capturing the model for CUDA graphs. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI.
INFO 07-23 23:46:41 model_runner.py:941] CUDA graphs can take additional 1~3 GiB memory per GPU. If you are running out of memory, consider decreasing `gpu_memory_utilization` or enforcing eager mode. You can also reduce the `max_num_seqs` as needed to decrease memory usage.
INFO 07-23 23:46:45 model_runner.py:1017] Graph capturing finished in 4 secs.
INFO 07-23 23:46:46 block_manager_v1.py:246] Automatic prefix caching is enabled.


In [2]:
def llm_pp_inf(text_mid):
    text = text_start + text_mid + text_end
    response = llm.generate(text, sampling_params)
    res = response[0].outputs[0].text.strip()
    return res
    #result = response[0].outputs[0].text
    #text_prosody = result.strip("Answer: ")
    #print(text_prosody)

In [14]:
text = '做运动是一件有益的事情'
res = llm_pp_inf(text)
print(res)

Processed prompts: 100%|██████████| 1/1 [00:00<00:00,  2.00it/s]

zuo4 #1 vn4 dong4 #1 shiii4 #1 i2 jian4 #1 iou3 i4 de5 #1 shiii4 qing5 #3
